In [4]:
import torch
import torchvision.utils as vutils
from torch import nn
ngf = 64
nc=3

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
            return output
# number of gpu's available
ngpu = 1
# input noise dimension
nz = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 1
netG = Generator(ngpu).to(device)
#load weights to test the model
netG.load_state_dict(torch.load('weights/netG_epoch_24.pth'))
trial = 1
noise1 = torch.randn(batch_size, nz, 1, 1, device=device)
noise2 = torch.randn(batch_size, nz, 1, 1, device=device)
fake_1 = netG(noise1)
fake_2 = netG(noise2)
vutils.save_image(fake_1.detach(),'output_new/test_3/fake_samples1%03d.png' % (trial),normalize=True)
vutils.save_image(fake_2.detach(),'output_new/test_3/fake_samples2%03d.png' % (trial),normalize=True)
for alpha in torch.linspace(0, 1, steps=10):  # 10 interpolation steps
    interpolated_noise = (1 - alpha) * noise1 + alpha * noise2
    
    interpolated_fake_image = netG(interpolated_noise)
    vutils.save_image(interpolated_fake_image.detach(),'output_new/test_3/interpolate_fake_sample_trial_%03d.png' % (trial),normalize=True)
    trial = trial + 1


In [5]:
torch.linspace(0, 1, steps=10)

tensor([0.0000, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889,
        1.0000])